In [2]:
import folium
import pandas as pd

In [ ]:
def adicionar_marcadores(grupo_modalidade, df):
    for _, row in df.iterrows():
        # Conteúdo HTML do popup
        popup_content = f"""
        <div style="
            border: 1px solid #ddd; 
            padding: 10px; 
            font-family: Arial, sans-serif; 
            font-size: 13px; 
            color: #333;
            border-radius: 6px;
            box-shadow: 0px 2px 5px rgba(0, 0, 0, 0.15);
            max-width: 250px;
        ">
            <h4 style="margin-top: 0; color: #4CAF50; font-size: 14px;">{row['Descrição']}</h4>
            <p style="margin: 5px 0;"><strong>Endereço:</strong> {row['Endereço']}, {row['Bairro']}</p>
            <p style="margin: 5px 0;"><strong>Cidade/UF:</strong> {row['Cidade']} - {row['UF']}</p>
            <p style="margin: 5px 0;"><strong>Preço:</strong> R$ {row['Preço']}</p>
            <p style="margin: 5px 0;"><strong>Desconto:</strong> {row['Desconto']}%</p>
            <p style="margin: 5px 0;">
                <a href="{row['Link de acesso']}" target="_blank" style="color: #1E90FF; text-decoration: none;">Mais informações</a>
            </p>
            <button onclick="saveProperty({row['N° do imóvel']})" style="
                background-color: #4CAF50; 
                color: white; 
                padding: 5px 10px; 
                font-size: 13px; 
                border: none; 
                border-radius: 4px; 
                cursor: pointer;
                margin-top: 5px;
            ">Salvar Imóvel</button>
        </div>
        """
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=folium.Popup(popup_content, max_width=300)
        ).add_to(grupo_modalidade)

def gera_mapa(df):
    # Script JavaScript para salvar imóvel
    save_script = """
    <script>
        function saveProperty(imovelId) {
            fetch('/save_property', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ id: imovelId }),
            }).then(response => {
                if (response.ok) {
                    alert('Imóvel salvo com sucesso!');
                } else {
                    alert('Erro ao salvar imóvel.');
                }
            });
        }
    </script>
    """
    
    # Define o centro do mapa com base na média das coordenadas
    centro_lat = df['Latitude'].mean()
    centro_lon = df['Longitude'].mean()
    mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=8)

    # Criação de grupos de imóveis filtrados por modalidade
    modalidades = df['Modalidade de venda'].unique()

    for modalidade in modalidades:
        grupo_modalidade = folium.FeatureGroup(name=f"Modalidade: {modalidade}")
        mapa.add_child(grupo_modalidade)

        # Filtra os imóveis por modalidade
        df_modalidade = df[df['Modalidade de venda'] == modalidade]

        # Adiciona os marcadores de imóveis dessa modalidade
        adicionar_marcadores(grupo_modalidade, df_modalidade)

    # Adiciona controle de camadas para o usuário escolher a modalidade de venda
    folium.LayerControl(collapsed=False).add_to(mapa)
    
    # Adiciona o script JavaScript ao mapa
    mapa.get_root().html.add_child(folium.Element(save_script))

    # Salva o mapa em um arquivo HTML
    mapa.save("../html/mapa_imoveis_leilao.html")
    print("Mapa gerado e salvo como 'mapa_imoveis_leilao.html'")



In [28]:
df = pd.read_csv('../data/lista_imoveis_sc_coordenadas.csv')
df.head()

,Unnamed: 0,N° do imóvel,UF,Cidade,Bairro,Endereço,Preço,Valor de avaliação,Desconto,Descrição,Modalidade de venda,Link de acesso,Rua,Número,Latitude,Longitude
0,0,8444426132271,SC,AGUAS MORNAS,SANTA CRUZ DA FIGUEIRA,"RUA ENGELBERTO KIRCHNER NETO, N. 58, CASA 01","230.000,00","230.000,00",0.00,"Casa, 0.00 de área total, 83.40 de área privat...",Leilão SFI - Edital Único,https://venda-imoveis.caixa.gov.br/sistema/det...,RUA ENGELBERTO KIRCHNER NETO,58,-27.721312,-48.884032
1,1,1444406396097,SC,ARAQUARI,ITINGA,"RUA MONACO, N. 355, LT 55 QD C","110.670,00","150.000,00",26.22,"Casa, 0.00 de área total, 59.86 de área privat...",Licitação Aberta,https://venda-imoveis.caixa.gov.br/sistema/det...,RUA MONACO,355,-26.416983,-48.780923
2,2,1444408140111,SC,ARARANGUA,HERCILIO LUZ,"RUA PEDRO JOAO FIGUEIREDO, N. 52, 0","181.315,92","284.500,00",36.27,"Casa, 0.00 de área total, 199.60 de área priva...",Venda Direta Online,https://venda-imoveis.caixa.gov.br/sistema/det...,RUA PEDRO JOAO FIGUEIREDO,52,-28.911928,-49.359711
3,3,1444416087011,SC,ARARANGUA,JARDIM DAS AVENIDAS,"AVENIDA GETULIO VARGAS, N. 1851, Apto 404, BL 05","436.839,65","400.000,00",0.00,"Apartamento, 0.00 de área total, 134.90 de áre...",Leilão SFI - Edital Único,https://venda-imoveis.caixa.gov.br/sistema/det...,AVENIDA GETULIO VARGAS,1851,-28.944062,-49.472085
4,4,10141322,SC,ASCURRA,ASCURRA,"RUA DANTE ZONTA (ANTIGA RUA PM-65), N. 281","1.015.000,00","2.030.000,00",50.00,"Galpão, 0.00 de área total, 1818.00 de área pr...",Venda Direta Online,https://venda-imoveis.caixa.gov.br/sistema/det...,RUA DANTE ZONTA (ANTIGA RUA PM-65),281,-26.956997,-49.376775


In [29]:
gera_mapa(df)

Mapa gerado e salvo como 'mapa_imoveis_leilao.html'
